In [ ]:
import livef1
from livef1.utils.constants import interpolation_map

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

session = livef1.get_session(season=2024, meeting_identifier="Spa", session_identifier="Race")

In [ ]:
from livef1.adapters import livetimingF1_request, livetimingF1_getdata
from urllib.parse import urljoin
session.get_topic_names()

data = livetimingF1_getdata(
    urljoin(session.full_path, session.topic_names_info["CarData.z"]["StreamPath"]),
    stream=True
)

In [ ]:
from livef1.utils.helper import *

sessionKey = 1
if isinstance(data, list):
    for ts, v in data:
        print(ts)
        parsed_entry = parse(v, zipped=True)
        for entry in parsed_entry["Entries"]:
            utc = entry["Utc"]
            for driver_entry in entry["Cars"].items():
                ch = driver_entry[1]["Channels"]
                record = {
                    "SessionKey": sessionKey,
                    "timestamp": ts,
                    "Utc": utc,
                    "DriverNo": driver_entry[0],
                    **{channel_name_map[k]:v for k,v in ch.items()}
                }
                print(record)

elif isinstance(data, str):
    parsed_entry = parse(data, zipped=True)
    for entry in parsed_entry["Entries"]:
        utc = entry["Utc"]
        for driver_entry in entry["Cars"].items():
            record = {
                "SessionKey": sessionKey,
                "timestamp": None,
                "Utc": utc,
                "DriverNo": driver_entry[0],
                **{channel_name_map[k]:v for k,v in ch.items()}
            }
            print(record)


In [ ]:
res = session.get_data(dataName="Position")
df_pos = pd.DataFrame(res.value)
df_pos["Utc"] = pd.to_datetime(df_pos["Utc"])
df_pos["timestamp"] = pd.to_datetime(df_pos["timestamp"])
df_pos["min"] = df_pos["Utc"].dt.minute
df_pos["hour"] = df_pos["Utc"].dt.hour
df_pos["sec"] = df_pos["Utc"].dt.second

In [ ]:
res = session.get_data(dataName="Car_Data")
df_car = pd.DataFrame(res.value)
df_car["Utc"] = pd.to_datetime(df_car["Utc"])
df_car["timestamp"] = pd.to_datetime(df_car["timestamp"])
df_car["min"] = df_car["Utc"].dt.minute
df_car["hour"] = df_car["Utc"].dt.hour
df_car["sec"] = df_car["Utc"].dt.second

In [ ]:
df_car.columns.tolist()

In [5]:
# display(df_pos[df_pos['DriverNo'] == "44"].iloc[168:1000].head(10))
# display(df_car[df_car['DriverNo'] == "44"].head(10))

# df_44 = df_pos[df_pos['DriverNo'] == "44"].reset_index()
# df_44["time_diff"] = (df_44["Utc"] - df_44["Utc"].shift(1)).dt.total_seconds()
# df_44[df_44["time_diff"] < 0.1]
# df_44.loc[108:118]
df = df_car.set_index("Utc").join(df_pos.set_index("Utc"), rsuffix="_pos", how="outer")

In [ ]:
df_44 = df[(df["DriverNo"] == "44") | (df["DriverNo_pos"] == "44")]
df_44

In [ ]:
df_44["X"] = df_44.reset_index()["X"].astype(float).interpolate(method = "quadratic", order = 2).values
df_44["Y"] = df_44.reset_index()["Y"].astype(float).interpolate(method = "quadratic", order = 2).values

In [ ]:
from plotly import express as px
df_temp = df[(df["DriverNo"] == "44") | (df["DriverNo_pos"] == "44")].iloc[22000:23500]
px.scatter(x = df_temp.X, y = df_temp.Y)

In [ ]:
df_temp = df_44.iloc[22000:23900][~df_44["SessionKey"].isnull()]
px.scatter(x = df_temp.X, y = df_temp.Y)

In [ ]:
df_temp = df_44.iloc[22000:23900]
px.scatter(x = df_temp.X, y = df_temp.Y)

In [ ]:
df_44["speed"] = df_44.reset_index()["speed"].astype(float).interpolate(method = "polynomial", order = 2).values
df_temp = df_44.iloc[22000:23500]
df_temp["fill"] = df_temp["DriverNo_pos"].isnull()
px.scatter(df_temp, x = df_temp.index, y = "speed", color="fill")

In [ ]:
res = session.get_data(dataName="Weather_Data")
df_weather = pd.DataFrame(res.value)
df_weather["timestamp"] = pd.to_datetime(df_weather["timestamp"])

In [7]:
idx_cols = ["hour","min","sec","DriverNo"]
df = df_car.set_index(idx_cols).join(df_pos.set_index(idx_cols), rsuffix="_")

In [ ]:
c = pd.to_datetime(df_car[(df_car["min"] == 2) & (df_car["hour"] == 0)].timestamp).unique()
p = pd.to_datetime(df_w[(df_w["min"] == 2) & (df_w["hour"] == 0) & (df_w["sec"] > 27)].timestamp).unique()

for i in c-p:
    print(i.total_seconds())

In [8]:
df_all = df.reset_index().set_index("timestamp").join(df_weather.set_index("timestamp"), rsuffix="_w_", how="outer")

In [ ]:
df_car[df_car["DriverNo"] == "44"]

In [ ]:
df.reset_index()[df.reset_index()["DriverNo"] == "44"]

In [ ]:
df_all[df_all["DriverNo"] == "44"].reset_index().iloc[44000:50000]

## Weather Data

In [9]:
for col in interpolation_map:
    df_all[col] = df_all.reset_index()[col].astype(float).interpolate(method = interpolation_map[col], order = 2).values

In [ ]:
import matplotlib.pyplot as plt

df_temp = df_all[~df_all["DriverNo"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.index), df_temp["AirTemp"])
df_temp = df_all[df_all["DriverNo"].isnull()]
plt.scatter(pd.to_datetime(df_temp.index), df_temp["AirTemp"])
plt.show()

In [ ]:
df_all.dropna(subset="DriverNo")

## Position Data

In [ ]:
df_all.dtypes

In [ ]:
df_temp = df_all[~df_all["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.index), df_temp["speed"])
df_temp = df_all[df_all["X"].isnull()]
plt.scatter(pd.to_datetime(df_temp.index), df_temp["speed"])
plt.show()

In [ ]:
df_44 = df_all[df_all["DriverNo"] == "44"].reset_index().iloc[44000:50000]

df_temp = df_44[~df_44["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["speed"])

df_temp = df_44[df_44["X"].isnull()]
plt.scatter(df_temp.index, df_temp["speed"])
plt.show()


df_temp = df_44[~df_44["X"].isnull()]
plt.figure(figsize=(20,5))
plt.scatter(pd.to_datetime(df_temp.timestamp), df_temp["speed"])
plt.show()

In [ ]:
pd.to_datetime(df_44.timestamp).dt.time - pd.to_datetime(df_44.timestamp).shift(1).dt.time

In [ ]:
df_44.head(50)

In [ ]:
(pd.to_datetime(df_44.timestamp) - pd.to_datetime(df_44.timestamp).shift(1)).dt.total_microseconds()

In [ ]:
df_44

In [ ]:
for col in df_weather.columns[2:]:
    plt.figure(figsize=(20,5))
    plt.scatter(pd.to_datetime(df_all.index).values, df_all[col].astype(float), label = col)
    plt.legend()
    plt.show()

In [ ]:
na_list = [np.nan] * 100
test_ts = pd.Series([-1] + na_list + [-1] + na_list + [-1] + na_list + [0] + na_list + [1]+ na_list + [1] + na_list + [1])

plt.figure(figsize=(20,5))
plt.scatter(test_ts.index, test_ts, label = "plain")
plt.plot(test_ts.index, test_ts.interpolate(method = "linear"), label = "linear")
plt.show()

for method in ['nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric', 'polynomial']:
    plt.figure(figsize=(20,5))
    plt.scatter(test_ts.index, test_ts, label = "plain")
    for d in [2,3,5]:
        plt.plot(test_ts.index, test_ts.interpolate(method = method, order = d), label = method + "_" + str(d))
    plt.legend()
    plt.show()

for method in ['krogh', 'piecewise_polynomial', 'spline', 'pchip', 'akima', 'cubicspline']:
    plt.figure(figsize=(20,5))
    plt.scatter(test_ts.index, test_ts, label = "plain")
    for d in [2,3,5]:
        plt.plot(test_ts.index, test_ts.interpolate(method = method, order = d), label = method + "_" + str(d))
    plt.legend()
    plt.show()

# plt.figure(figsize=(20,5))
# for d in [2,3,5]:
#     plt.plot(test_ts.index, test_ts.interpolate(method = "spline", order = d), label = "spline_" + str(d))
# plt.legend()
# plt.show()



In [ ]:
df_temp = df_all.copy()
df_temp.index = pd.to_datetime(df_all.index).values

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "linear"), label = "linear")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "polynomial", order = 2), label = "polynomial")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float).interpolate(method = "spline", order = 2), label = "spline")
plt.show()

plt.figure(figsize=(20,5))
plt.scatter(df_temp.index, df_temp["WindSpeed"].astype(float), label = "plain")
plt.show()



In [ ]:
len(df_temp)
df_viz

In [ ]:
for d in [2,3,5,7,9,15,25,35]:
    try:
        df_viz = df_temp["WindSpeed"].astype(float).interpolate(method = "linear", order = d, limit_direction="both").reset_index().iloc[500000:1000000]
        print(d)
        plt.figure(figsize=(20,5))
        plt.scatter(df_viz["index"], df_viz["WindSpeed"], label = str(d))
        plt.show()
    except:
        pass

In [ ]:
df_all.reset_index()[~df_all.reset_index()["AirTemp"].isnull()]

In [ ]:
df_all.reset_index().iloc[2373112-25:2373112+25]

In [14]:
import numpy as np

cols = df_weather.columns[2:]
df_all.loc[:,cols] = df_all.loc[:,cols].ffill()


In [ ]:
df_all.dropna(subset="Channels").dtypes

In [ ]:
df_all

In [ ]:
res = session.get_data(dataName="Weather_Data")
df_w = pd.DataFrame(res.value)

In [ ]:
res = session.get_data(dataName="Track_Status")
df = pd.DataFrame(res.value)
# df["Utc"] = pd.to_datetime(df["Utc"])
# df["duration"] = df["Utc"] - df["Utc"].shift(1)
df

In [ ]:
res = session.get_data(dataName="Tyre_Stints")
df = pd.DataFrame(res.value)
df

In [ ]:
df

In [ ]:
res = session.get_data(dataName="Weather_Data")
df = pd.DataFrame(res.value)
df

In [ ]:
from livef1.adapters.livetimingf1_adapter import livetimingF1_getdata

data = livetimingF1_getdata("https://livetiming.formula1.com/static/2024/2024-07-28_Belgian_Grand_Prix/2024-07-28_Race/PitLaneTimeCollection.jsonStream", stream=True)
data

for key, value in data.items():
    if "_deleted" in value["PitTimes"].keys():
        for deleted_driver in value["PitTimes"]["_deleted"]:
            record = {
                "session_key": 0,
                "timestamp": key,
                "_deleted": deleted_driver
            }
            print(record)
            
    else:
        for driver_no, info in value["PitTimes"].items():
            print(info)
            record = {
                "session_key": 0,
                "timestamp": key,
                **info
            }
            print(record)
    
    print("----------")

In [ ]:
res = session.get_data(dataName="Pit_Lane_Times")
df = pd.DataFrame(res.value)
df

In [ ]:
import fastf1

session = fastf1.get_session(2024, 'Belgian', 'Race')
session.load()

In [ ]:
help(session)

In [ ]:
session.weather_data

In [ ]:
session.laps

In [ ]:
session.laps[~session.laps["PitInTime"].isnull()].sort_values("Time")

In [ ]:
session.laps[~session.laps["PitOutTime"].isnull()].sort_values("Time")

In [ ]:
session.race_control_messages

In [ ]:
session.results

In [ ]:
session.car_data["1"]